# Assignment 2

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
from patsy import dmatrices
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [9]:
# DATA IMPORT - FROM GITHUB
data_raw = pd.read_csv('https://raw.githubusercontent.com/thimaipham/2.CEU_DA3_Assignment2/main/listings.csv')

### 1. Data Featuring

##### Data describe:
- The raw dataset comprises 25,480 observations and 76 columns, including some columns that are unnecessary or difficult to use during the modeling process.
- Certain important columns are encountering the following issues:
    1. The `bedrooms` column has a staggering 99% missing values.
    2.The `name` column contains descriptions of apartments, including the number of bedrooms.
    3. The `bathrooms` column has no data.
The bathroom_text column has data but is currently of object type due to the presence of the word 'bath' in the entries.
The price column is currently of object type as it contains the '$' symbol.
Several columns exhibit a high percentage of missing values (ranging from 40% to 90%).The raw dataset comprises 25,480 observations and 76 columns, including some columns that are unnecessary or difficult to use during the modeling process.
Certain important columns are encountering the following issues:
The bedrooms column has a staggering 99% missing values.
The name column contains descriptions of apartments, including the number of bedrooms.
The bathrooms column has no data.
The bathroom_text column has data but is currently of object type due to the presence of the word 'bath' in the entries.
The price column is currently of object type as it contains the '$' symbol.
Several columns exhibit a high percentage of missing values (ranging from 40% to 90%).

In [85]:
# Regex looks for a number followed by the word 'bedroom' or 'bedrooms'
data_raw['bedroom_extract'] = data_raw['name'].str.extract('(\d+) bedroom')

# Convert the extracted bedroom numbers to numeric type
data_raw['bedroom_extract'] = pd.to_numeric(data_raw['bedroom_extract'], errors='coerce')

In [95]:
# Removing unnecessary columns to streamline the dataset for this assignment
# List of columns to be dropped based on the description and analysis
columns_to_drop = [
    'id', 'listing_url', 'scrape_id', 'name', 'last_scraped', 'description',
    'neighborhood_overview', 'picture_url', 'host_id', 'host_url','bedrooms','host_name', 
    'host_thumbnail_url', 'host_picture_url', 'host_about', 'host_neighbourhood',
    'host_listings_count', 'host_total_listings_count', 'neighbourhood', 
    'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'calendar_last_scraped', 
    'license', 'host_location', 'source','first_review','last_review','host_verifications',
    'calendar_updated', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
    'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 
    'amenities' # amenities column is emty with [] symbol
]

# Dropping the columns from the DataFrame
data_cleaned = data_raw.drop(columns=columns_to_drop, errors='ignore')

# We focus on apartments with 2<= n <= 6
data_cleaned = data_cleaned[(data_cleaned.accommodates >= 2) & (data_cleaned.accommodates <= 6)]

# Removing rows with null values in the 'price' variable for accurate modeling
data_cleaned = data_cleaned.dropna(subset=['price'])


In [88]:
# Convert 'bathrooms_text' to a numeric 'bathrooms' column by extracting the number
data_cleaned['bathrooms'] = data_cleaned['bathrooms_text'].str.extract('(\d+)').astype(float)

# Convert 'price' to a numeric column by removing the '$' and ',' then converting to float
data_cleaned['price'] = data_cleaned['price'].replace('[\$,]', '', regex=True).astype(float)

# Drop the original 'bathrooms_text' column as it's now redundant
data_cleaned = data_cleaned.drop('bathrooms_text', axis=1)



In [96]:
# Calculate the percentage of missing values for each column
missing_percentage = data_cleaned.isnull().mean() * 100

# Due to some columns which have more than 40% of missing values, I will drop those columns
# Identify columns with more than 40% missing values
columns_to_drop_missing = missing_percentage[missing_percentage > 40].index

# Drop these columns
data_cleaned = data_cleaned.drop(columns=columns_to_drop_missing)

# Drop observations with any missing values
data_cleaned = data_cleaned.dropna()


##### After cleaning part the cleaned data includes: 
- 13773 observations
- 38 columns